In [1]:
import argparse
from collections import defaultdict, OrderedDict
import logging
import numpy as np

import dist_ir
from dist_ir.importer import import_from_onnx, parse_tensor_from_file
from dist_ir.ir import FunctionMaker, cpprint, pformat, Device, Topology, Value
from dist_ir.executor import infer_types, SequentialExecutor, Simulator
from dist_ir.executor.cost_model import CostModel
from dist_ir.ir.type import Bool, Float, Int64, Tensor
from dist_ir.transforms import (
    parallel_transform_3d,
    hybrid_transform_unrolled,
    PipeDreamScheduler,
)

In [2]:
device_speeds = {"gpu": 1.0e13}

In [3]:
topology = Topology(device_speeds)
d0 = topology.add_device("gpu")
function = FunctionMaker()
x = function.add_input_value("x", Tensor(Float(), shape=(1024, 512), device=d0))
w = function.add_input_value("w", Tensor(Float(), shape=(512, 512), device=d0))
y = function.add_op("MatMul", inputs=[x, w], output_names=["y"])
function = function.finalize()
function = infer_types(function, function.inputs)
simulator = Simulator(CostModel(topology))
simulation = simulator.interpret(function, (v.type for v in function.inputs))
running_time = max(simulation.timestamps[d] for d in simulation.timestamps)
print(f"Running time: {running_time * 1e3} ms")

Running time: 0.0536870912 ms


In [4]:
topology = Topology(device_speeds)
d0 = topology.add_device("gpu")
d1 = topology.add_device("gpu")
topology.set_bandwidth(d0, d1, 200.0 * 8)
function = FunctionMaker()
y = function.add_input_value("y", Tensor(Float(), shape=(1024, 512), device=d0))
y1 = function.add_op("Send", inputs=[y], attributes={"device": d1}, output_names=["y1"])
function = function.finalize()
function = infer_types(function, function.inputs)
simulator = Simulator(CostModel(topology))
simulation = simulator.interpret(function, (v.type for v in function.inputs))
running_time = max(simulation.timestamps[d] for d in simulation.timestamps)
print(f"Running time: {running_time * 1e3} ms")

Running time: 0.00065536 ms


In [5]:
topology = Topology(device_speeds)
d0 = topology.add_device("gpu")
d1 = topology.add_device("gpu")
topology.set_bandwidth(d0, d1, 200.0 * 8)
function = FunctionMaker()
y0 = function.add_input_value("y0", Tensor(Float(), shape=(1024, 512), device=d0))
y1 = function.add_input_value("y1", Tensor(Float(), shape=(1024, 512), device=d1))
y1 = function.add_op("MPIAllreduce", inputs=[y0, y1], output_names=["y1"])
function = function.finalize()
function = infer_types(function, function.inputs)
simulator = Simulator(CostModel(topology))
simulation = simulator.interpret(function, (v.type for v in function.inputs))
running_time = max(simulation.timestamps[d] for d in simulation.timestamps)
print(f"Running time: {running_time * 1e3} ms")

> /Users/keshavsanthanam/workspace/dist-ir/dist_ir/executor/cost_model.py(145)_mpi_allreduce_cost_fn()
    143 
    144         pdb.set_trace()
--> 145         input_size = xs[0].size()
    146         devices = [x.device for x in xs]
    147         num_devices = len(devices)

ipdb> n
> /Users/keshavsanthanam/workspace/dist-ir/dist_ir/executor/cost_model.py(146)_mpi_allreduce_cost_fn()
    144         pdb.set_trace()
    145         input_size = xs[0].size()
--> 146         devices = [x.device for x in xs]
    147         num_devices = len(devices)
    148         per_device_data = 2 * input_size * (num_devices - 1) / num_devices

ipdb> n
> /Users/keshavsanthanam/workspace/dist-ir/dist_ir/executor/cost_model.py(147)_mpi_allreduce_cost_fn()
    145         input_size = xs[0].size()
    146         devices = [x.device for x in xs]
--> 147         num_devices = len(devices)
    148         per_device_data = 2 * input_size * (num_devices - 1) / num_devices
    149         per_device_data_

ipdb> n
> /Users/keshavsanthanam/workspace/dist-ir/dist_ir/executor/cost_model.py(154)_mpi_allreduce_cost_fn()
    152             for j in range(i + 1, len(devices)):
    153                 all_bandwidths.append(
--> 154                     self._topology.get_bandwidth(devices[i], devices[j])
    155                 )
    156         average_bandwidth = np.mean(all_bandwidths)

ipdb> n
> /Users/keshavsanthanam/workspace/dist-ir/dist_ir/executor/cost_model.py(153)_mpi_allreduce_cost_fn()
    151         for i in range(len(devices)):
    152             for j in range(i + 1, len(devices)):
--> 153                 all_bandwidths.append(
    154                     self._topology.get_bandwidth(devices[i], devices[j])
    155                 )

ipdb> average_bandwidth
*** NameError: name 'average_bandwidth' is not defined
ipdb> n
> /Users/keshavsanthanam/workspace/dist-ir/dist_ir/executor/cost_model.py(152)_mpi_allreduce_cost_fn()
    150         all_bandwidths = []
    151         for i 

ipdb> cost
1.6384000000000002e-07
ipdb> n
--Return--
{Device(device...ariable=False): 1.6384000000000002e-07, Device(device...ariable=False): 1.6384000000000002e-07}
> /Users/keshavsanthanam/workspace/dist-ir/dist_ir/executor/cost_model.py(159)_mpi_allreduce_cost_fn()
    157         cost = per_device_data_gb / average_bandwidth
    158 
--> 159         return {device: cost for device in devices}
    160 
    161     def _mpi_broadcast_cost_fn(self, op, x):

ipdb> n
> /Users/keshavsanthanam/workspace/dist-ir/dist_ir/executor/simulator.py(122)semantics()
    120             costs = cost_fn(op, *inputs)
    121 
--> 122             if not isinstance(outputs, tuple):
    123                 outputs = (outputs,)
    124             for x, val in zip(op.outputs, outputs):

ipdb> n
> /Users/keshavsanthanam/workspace/dist-ir/dist_ir/executor/simulator.py(124)semantics()
    122             if not isinstance(outputs, tuple):
    123                 outputs = (outputs,)
--> 124             for 

ipdb> c
Running time: 0.00016384 ms
